In [4]:
import sys
import os
import faiss
os.environ['HF_HOME']='/hdd/'
from datasets import load_from_disk

In [5]:
import logging
logging.basicConfig(level=20)


In [3]:
kilt_wiki = load_from_disk("/hdd/kilt")

In [6]:
kilt_wiki

Dataset({
    features: ['embeddings', 'text', 'title', 'wikipedia_id'],
    num_rows: 29167229
})

In [7]:
import numpy as np
np.sqrt(len(kilt_wiki))

5400.669310372557

In [10]:
D = 768

# Param of PQ
M = 64  # The number of sub-vector.
nbits = 8 # bits per sub-vector. This is typically 8, so that each sub-vec is encoded by 1 byte

# Param of IVF
nlist = 1000  # The number of cells (space partition), according to the quantizer. Typical value is sqrt(N)
# Param of HNSW
hnsw_m = 32  # The number of neighbors for HNSW. This is typically 32

# Setup
quantizer = faiss.IndexHNSWFlat(D, hnsw_m)
index = faiss.IndexIVFPQ(quantizer, D, nlist, M, nbits, faiss.METRIC_INNER_PRODUCT)

kilt_wiki.add_faiss_index(column='embeddings', custom_index=index, train_size=100_000)


Dataset({
    features: ['embeddings', 'text', 'title', 'wikipedia_id'],
    num_rows: 29167229
})

In [14]:
kilt_wiki.save_faiss_index('embeddings', file='/hdd/kilt/index_v0.faiss') # 2GB. 48x

In [8]:
# TODO: SANITY CHECK
65536 / 2e8

0.00032768

In [9]:
# https://github.com/facebookresearch/faiss/wiki/The-index-factory
# https://github.com/facebookresearch/faiss/wiki/Indexing-1G-vectors
# https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index
# TODO: How to METRIC_INNER_PRODUCT?
# Intended to do similar to above, but with dim reduction at the start.
# preprocess- 128 is the output dimension. 
# clustering - coarse quantizer (first parameter of the constructor) should be. quantizer, and nlist (16384)
# Product Quantization codes.  M=16, nbits=12
train_size = 30 * 65536/2 # per https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index
string_index = "OPQ64_128,IVF65536_HNSW32,PQ64x8"
kilt_wiki.add_faiss_index(column='embeddings', string_factory=string_index, train_size=train_size)

TypeError: slice indices must be integers or None or have an __index__ method

In [1]:
kilt_wiki.save_faiss_index('embeddings', file='/hdd/kilt/index_v1.faiss') # 2GB. 48x

NameError: name 'kilt_wiki' is not defined

In [12]:
#https://discuss.huggingface.co/t/runtimeerror-error-in-void-faiss-allocmemoryspace/1358/8
# import faiss

# d = 768  # vectors dimension
# m = 128  # hnsw parameter. Higher is more accurate but takes more time to index (default is 32, 128 should be ok)
# index = faiss.IndexHNSWFlat(d, m, faiss.METRIC_INNER_PRODUCT)

In [1]:
import numpy as np
D = 768
quantizer = faiss.IndexHNSWFlat(D, hnsw_m)
index = faiss.IndexIVFPQ(quantizer, D, nlist, M, nbits)

In [2]:
np.array([kilt_wiki[i]['embeddings'] for i in range(2045,2050)])

NameError: name 'kilt_wiki' is not defined

In [9]:
# https://github.com/facebookresearch/DPR/issues/6 
# 21MM -> 60GB
29_167_229 * 4 * 768 / 10e9

8.9601727488

In [11]:
32*768/(8*64)

48.0